In [ ]:
from getpass import getpass

# Prompt for token securely
token = getpass('Enter your GitHub personal access token: ')

# Clone using token authentication
!git clone https://{token}@github.com/pabasara-samarakoon-4176/MDT_prediction.git

In [17]:
# Developed by Shakya Rupasighe
import os
import sys
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
df = pd.read_csv('/content/Sri-Lanka.csv')

In [ ]:
df.head()

,Cell_ID,Cell Name,RSRP,RSRQ,Geohash,EARFCN_DL,Latitude_of_MDT,Longitude_of_MDT,MR_Count,Site ID,...,oneway,bridge,layer,source,name:si,name:ta,osm_id,osm_type,Road_Coverage,Population_Density
0,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,NaN,156298074.0,ways_line,1,3291.916504
1,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,NaN,197793816.0,ways_line,1,3291.916504
2,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,NaN,197793916.0,ways_line,1,3291.916504
3,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,NaN,197793699.0,ways_line,1,3291.916504
4,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,NaN,156298067.0,ways_line,1,3291.916504


In [ ]:
df['geometry'] = df.apply(lambda row: Point(row['Longitude_of_MDT'], row['Latitude_of_MDT']), axis=1)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

In [ ]:
building_data = gpd.read_file('/content/drive/MyDrive/Final_year_project/building/gis_osm_buildings_a_free_1.shp')
building_data = building_data.to_crs("EPSG:4326")

In [9]:
projected_crs = "EPSG:5234"
gdf = gdf.to_crs(projected_crs)
building_data = building_data.to_crs(projected_crs)

In [10]:
buffer_distance = 100
gdf['geometry'] = gdf.geometry.buffer(buffer_distance)

In [11]:
joined = gpd.sjoin(gdf, building_data, how="left", predicate="intersects")

In [12]:
building_counts = joined.groupby(joined.index).size()

In [13]:
gdf["Building_Density"] = gdf.index.map(building_counts).fillna(0).astype(int)

In [14]:
gdf = gdf.to_crs("EPSG:4326")

In [15]:
gdf = gdf.drop(columns=['geometry'])

In [16]:
gdf.head()

,Cell_ID,Cell Name,RSRP,RSRQ,Geohash,EARFCN_DL,Latitude_of_MDT,Longitude_of_MDT,MR_Count,Site ID,...,bridge,layer,source,name:si,name:ta,osm_id,osm_type,Road_Coverage,Population_Density,Building_Density
0,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,156298074.0,ways_line,1,3291.916504,67
1,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,197793816.0,ways_line,1,3291.916504,67
2,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,197793916.0,ways_line,1,3291.916504,67
3,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,197793699.0,ways_line,1,3291.916504,67
4,5215253,Mannar_SLT-MA0001-H_2B,-100.0,-13.0,tc8g466m,1650,8.978096,79.906923,2,MA0001,...,NaN,NaN,NaN,NaN,NaN,156298067.0,ways_line,1,3291.916504,67
